In [1]:
import os
import cv2
import sys
import glob
import json
import time
import random
import hashlib
import imutils
import argparse
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from collections import defaultdict

from core.detector import LFFDDetector

In [2]:
# Setting
detector_path = "configs/anime.json"
use_gpu = 0
size = None
confidence_threshold = None
nms_threshold = None
fps = None
run_per_x_frames = 120
frame_count = None

with open(detector_path, "r") as f:
    config = json.load(f)
if use_gpu > 0:
    use_gpu = True
else:
    use_gpu = False
detector = LFFDDetector(config, use_gpu=use_gpu)

Wed Mar 17 00:18:28 2021: Initializing.
Wed Mar 17 00:18:28 2021: Initialized (0.1126 seconds).


In [3]:
# 모든 데이터(영상)의 주소를 딕셔너리 형태로 리턴
def get_all_videos():
    d = {}
    root_path = glob("./data/*")
    for i in range(len(root_path)):
        root_path[i] = root_path[i].replace('\\', '/')
    for path in root_path:
        li = []
        for file_path in glob(f"{path}/*"):
            t = file_path.replace('\\', '/')
            if t not in li:
                li.append(t)
        li.sort()
        d[path+'/'] = li
    cnt = 0
    for file_path, file_names in d.items():
        print(f"{file_path}: {file_names}")
        cnt += len(file_names)
    print(f"\n{cnt} Videos found\n")

    return d

In [4]:
def face_detection(df, video_path, output_path, detector, size=None, confidence_threshold=None, nms_threshold=None, roi=(), frame_count=None, run_per_x_frames=120, fps=None):
    cap = cv2.VideoCapture(video_path)
    fc = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = fps or cap.get(cv2.CAP_PROP_FPS) / run_per_x_frames
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if roi:
        xmin, ymin, xmax, ymax = roi
        frame_width, frame_height = xmax - xmin, ymax - ymin
    else:
        xmin = 0
        ymin = 0
        xmax = frame_width
        ymax = frame_height
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fc = frame_count or fc
    frame_count = fc
    bar = tqdm(total=int(frame_count))

    if not os.path.exists(output_path):
        os.makedirs(output_path)
    cnt = len(glob(f"{output_path}*"))-1
    if cnt < 0:
        cnt += 1
    while True:
        ret = cap.grab()
        frame_idx = cap.get(cv2.CAP_PROP_POS_FRAMES)
        if not ret or frame_idx > frame_count:
            break
        if int(frame_idx) % run_per_x_frames != 0:
            bar.update()
            continue

        _, frame = cap.retrieve()
        boxes = detector.detect(frame, size=size, confidence_threshold=confidence_threshold, nms_threshold=nms_threshold)
        if boxes:
            s = ""
            for i in range(len(boxes)):
                xmin, ymin, xmax, ymax = boxes[i]["xmin"], boxes[i]["ymin"], boxes[i]["xmax"], boxes[i]["ymax"]
                s += f"{ymin}|{xmin}|{ymax}|{xmax}/"
            df.loc[len(df)] = [f"{output_path}{cnt}.jpg", s]
            cv2.imwrite(f"{output_path}/{cnt}.jpg", frame)
            cnt += 1
        bar.update()
    bar.close()
    cap.release()
    df.to_csv( f"{output_path}ROI.csv", encoding='euc-kr')
    
    return cnt

In [5]:
# input_path = "data/Kaguya/Kaguya1.mp4"
# output_path = "result/Kaguya/"
# df = pd.DataFrame(columns=("image_path","ROI"))
# face_detection(df, input_path, output_path, detector, size=size,
#     confidence_threshold=confidence_threshold, nms_threshold=nms_threshold,
#     frame_count=frame_count, run_per_x_frames=run_per_x_frames, fps=fps
# )

In [6]:
# Processing All
d = get_all_videos()
for file_path, file_names in d.items():
    anime_name = file_path.split('/')[-2]
    output_path = f"./result/{anime_name}/"
    print(f"{anime_name} Start")
    df = pd.DataFrame(columns=("image_path","ROI"))
    for input_path in file_names:
        print("file_path:", file_path)
        print("file name:", input_path)
        print("output_path:", output_path, '\n')
        cnt = face_detection(df, input_path, output_path, detector, size=size,
        confidence_threshold=confidence_threshold, nms_threshold=nms_threshold,
        frame_count=frame_count, run_per_x_frames=run_per_x_frames, fps=fps
        )
        print(f"{anime_name} 총 {cnt}개 저장됨.\n")

  0%|▏                                                                             | 82/38148 [00:00<00:48, 783.05it/s]

./data/Kaguya/: ['./data/Kaguya/Kaguya1.mp4']
./data/PrincessConnectReDive/: ['./data/PrincessConnectReDive/PrincessConnectReDive1.mp4']
./data/ReZeroFrozenBonds/: ['./data/ReZeroFrozenBonds/ReZeroFrozenBonds.mkv']
./data/SteinsGate/: ['./data/SteinsGate/SteinsGate1.mp4']

4 Videos found

Kaguya Start
file_path: ./data/Kaguya/
file name: ./data/Kaguya/Kaguya1.mp4
output_path: result/Kaguya/ 



  0%|▏                                                                             | 60/34526 [00:00<01:02, 551.93it/s]

Kaguya 총 168개 저장됨.

PrincessConnectReDive Start
file_path: ./data/PrincessConnectReDive/
file name: ./data/PrincessConnectReDive/PrincessConnectReDive1.mp4
output_path: result/PrincessConnectReDive/ 



  0%|                                                                             | 30/114163 [00:00<06:26, 295.01it/s]

PrincessConnectReDive 총 174개 저장됨.

ReZeroFrozenBonds Start
file_path: ./data/ReZeroFrozenBonds/
file name: ./data/ReZeroFrozenBonds/ReZeroFrozenBonds.mkv
output_path: result/ReZeroFrozenBonds/ 



  0%|                                                                              | 47/34432 [00:00<01:16, 450.04it/s]

ReZeroFrozenBonds 총 308개 저장됨.

SteinsGate Start
file_path: ./data/SteinsGate/
file name: ./data/SteinsGate/SteinsGate1.mp4
output_path: result/SteinsGate/ 



100%|███████████████████████████████████████████████████████████████████████████| 34432/34432 [01:59<00:00, 288.21it/s]

SteinsGate 총 149개 저장됨.

